# AWS Sagemaker: Bring Your Own Model


This notebook shows how to dockerize **any machine learning algorithm** and deploy it in the cloud. It is written in conjunction with this [medium post](https://medium.com/p/92778dabb32f).

Let's change into the 'container' directory

In [1]:
cd container

/home/sam/github/amazon-sagemaker-byom/container


*that bash command worked in a Python notebook because it is [automagic](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html).*

Let's also import some code

In [2]:
import json 
from IPython.display import Image

from utils.get_model import download_ResNet50
from utils.name_check import name_check
from utils import local





## A trained model

Let's download a trained model. This example uses [ResNet50](https://arxiv.org/abs/1512.03385) trained on ImageNet. It is one of the Keras [available models](https://keras.io/applications/#available-models). You **can** use a framework other than Keras. Basically anything will run in a Docker container.

In [ ]:
download_ResNet50()

That downloaded *model.json* and *weights.hdf5*. These files represent the model architecture and trained weights. When you are ready you can replace them with your own trained model. Here is where they go.

In [3]:
ls model

model.json  weights.hdf5


## Build Docker image

Install Docker for [Windows](https://docs.docker.com/toolbox/toolbox_install_windows/) or [Mac](https://docs.docker.com/toolbox/toolbox_install_mac/). 

**Linux Install**

For Debian-based systems, copy and paste this into a separate terminal.

In [4]:
# $ sudo apt install docker.io

For any other distro copy and paste this:

In [5]:
# $wget -qO- https://get.docker.com/ | sh

for further help (if required) check the docs for [Ubuntu](https://docs.docker.com/v17.09/engine/installation/linux/docker-ce/ubuntu/), [Debian](https://docs.docker.com/v17.09/engine/installation/linux/docker-ce/debian/), [CentOS](https://docs.docker.com/v17.09/engine/installation/linux/docker-ce/centos/) or [Fedora](https://docs.docker.com/v17.09/engine/installation/linux/docker-ce/fedora/).

*Post-Installation*

For all Linux installtions **[add your user to a docker group](https://docs.docker.com/v17.09/engine/installation/linux/linux-postinstall/#manage-docker-as-a-non-root-user)** to avoid using sudo.

**Docker Build**

Let's name the Docker image we are about to build

In [6]:
name = 'imagnet'
tag = 'dev-1.0.0'

There are a bunch of restrictions imposed by Amazon and Docker on the name we can use. Long story short, chose a lower case, alphanumeric *name*. The *tag* has less retrictions.

Let's check the name is valid before investing time in building an image.

In [7]:
name_check(name)

'The name "imagnet" is valid'

Assuming the name is valid, let's build the image

In [8]:
!docker build . --tag {name}:{tag}

Sending build context to Docker daemon  103.1MB
Step 1/12 : FROM ubuntu:16.04
 ---> 7aa3602ab41e
Step 2/12 : MAINTAINER Sam Murphy <samsammurphy@gmail.com>
 ---> Running in 2386f8c08f8c
Removing intermediate container 2386f8c08f8c
 ---> 92c6cc4d9fb8
Step 3/12 : EXPOSE 8080
 ---> Running in 266fd071f338
Removing intermediate container 266fd071f338
 ---> 10f415ee91d0
Step 4/12 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Running in fb958cbb3535
Get:1 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:5 http://security.ubuntu.com/ubuntu xenial-security/universe Sources [85.7 kB]
Get:6 http://security.ubuntu.com/ubuntu xenial

debconf: delaying package configuration, since apt-utils is not installed
Fetched 19.1 MB in 7s (2575 kB/s)
Selecting previously unselected package libxau6:amd64.
(Reading database ... 4768 files and directories currently installed.)
Preparing to unpack .../libxau6_1%3a1.0.8-1_amd64.deb ...
Unpacking libxau6:amd64 (1:1.0.8-1) ...
Selecting previously unselected package libjpeg-turbo8:amd64.
Preparing to unpack .../libjpeg-turbo8_1.4.2-0ubuntu3.1_amd64.deb ...
Unpacking libjpeg-turbo8:amd64 (1.4.2-0ubuntu3.1) ...
Selecting previously unselected package libpython2.7-minimal:amd64.
Preparing to unpack .../libpython2.7-minimal_2.7.12-1ubuntu0~16.04.3_amd64.deb ...
Unpacking libpython2.7-minimal:amd64 (2.7.12-1ubuntu0~16.04.3) ...
Selecting previously unselected package python2.7-minimal.
Preparing to unpack .../python2.7-minimal_2.7.12-1ubuntu0~16.04.3_amd64.deb ...
Unpacking python2.7-minimal (2.7.12-1ubuntu0~16.04.3) ...
Selecting previously unselected package python-minimal.
Preparing t

Setting up libpython2.7-stdlib:amd64 (2.7.12-1ubuntu0~16.04.3) ...
Setting up python2.7 (2.7.12-1ubuntu0~16.04.3) ...
Setting up libpython-stdlib:amd64 (2.7.12-1~16.04) ...
Setting up python (2.7.12-1~16.04) ...
Setting up libjbig0:amd64 (2.1-3.1) ...
Setting up libidn11:amd64 (1.32-3ubuntu1.2) ...
Setting up libpng12-0:amd64 (1.2.54-1ubuntu1.1) ...
Setting up ucf (3.0036) ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Can't locate Term/ReadLine.pm in @INC (you may need to install the Term::ReadLine module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.22.1 /usr/local/share/perl/5.22.1 /usr/lib/x86_64-linux-gnu/perl5/5.22 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl/5.22 /usr/share/perl/5.22 /usr/local/lib/site_perl /usr/lib/x86_64-linux-gnu/perl-base .) at /usr/share/perl5/Debconf/FrontEnd/Rea

  1550K .......... .......... .......... .......... .......... 99% 2.95M 0s
  1600K ....                                                  100% 7678G=0.7s

2018-08-03 14:52:48 (2.10 MB/s) - 'get-pip.py' saved [1642522/1642522]

  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
  Running setup.py bdist_wheel for MarkupSafe: started
  Running setup.py bdist_wheel for MarkupSafe: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/33/56/20/ebe49a5c612fffe1c5a632146b16596f9e64676768661e4e46
Successfully built itsdangerous MarkupSafe
Removing intermediate container b36ad4b26f14
 ---> e105c90a9847
Step 6/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in 4addaef8b848
Removing intermediate container 4addaef8b848
 ---> e41cf502637f
Step 7/12 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Runni

  Running setup.py bdist_wheel for gast: started
  Running setup.py bdist_wheel for gast: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4c/16/ef/e36a23f2432e9220f8845f94e2c3abd39e7d9d1cd458d3159d
  Running setup.py bdist_wheel for termcolor: started
  Running setup.py bdist_wheel for termcolor: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for pyyaml: started
  Running setup.py bdist_wheel for pyyaml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built gast absl-py termcolor pyyaml
  Found existing installation: 

## Local test

Start a local webserver with the machine learning code and endpoints all set up

In [18]:
tag

'dev-1.0.0'

In [16]:
!./run_local_server.sh {name}:{tag}

db8f04a3f1b185ec19aac4c245b08fd03eede0068e1ec418ca699b3fd2115fe1


In [22]:
!docker run --name mylocalservice -d -p 8080:8080 --rm {name}:{tag} serve

38c77cf2e0111c3e50785b60bb258f27db196d32c992a186408d130e28602446


The /ping endpoint checks if the connection is up

# write python files to do this

In [21]:
local.get('http://localhost:8080/ping')

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /ping (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fca640a37f0>: Failed to establish a new connection: [Errno 111] Connection refused',))

Let's take a look at one of our images.

In [13]:
dog = '../images/dog.jpg'
# Image(dog) 

and see if ResNet can determine what it is..

In [14]:
local.post(dog, route='invocations')

requests.exceptions.ConnectionError(urllib3.exceptions.MaxRetryError("HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fca64097978>: Failed to establish a new connection: [Errno 111] Connection refused',))"))

ok, but can we can identify if something is a hotdog?

In [ ]:
hotdog = '../images/hotdog.jpg'
# Image(hotdog) 

In [ ]:
local.post(hotdog, route='invocations')

Yes, it turns out we can, assuming all went well up to this point.

When you are done with local testing, kill the container running in the background.

In [15]:
!docker kill mylocalservice

Error response from daemon: Cannot kill container: mylocalservice: No such container: mylocalservice


## Register the container

The aim here is to send a docker image to Amazon's Elastic Container Registery (ECR). It seems that Amazon are using 'image' and 'container' interchangeably. 

I am obliged to say they are not the same thing; i.e. a container is an instance of an image. 

*Aside: However, I understand why Amazon might do this. The official Docker terminology here is not ideal. Although the term 'image' has been used for a [long time](https://en.wikipedia.org/wiki/Disk_image) in computing to describe content storage, for most people the word conjures up thoughts of something like a photo, which is confusing. In my humble opinion **Docker** should haved used the term 'container' instead of image and used 'instance' instead of container.*

**Ship the image to Amazon ECR**

Let's use the [AWS command line tool](https://docs.aws.amazon.com/cli/latest/userguide/installing.html) to send the docker image to the cloud.

In [ ]:
# To install and configure AWS run these commands in a separate terminal.
# $ pip install awscli
# $ aws configure

In [ ]:
!./send_to_Amazon.sh {name}

If you would like to see it in the Amazon Console look for the 'Elastic Container Service' (ECS).

## Host on SageMaker

The plan here is to host the machine learning algorithm on SageMaker using the docker image that we sent to ECR previously.

**IAM** 

SageMaker will need permission to run a hosting service. As this is really important but not necessarily obvious I will describe how I created my role. I started at the url produced by the code in the next cell.

In [ ]:
region = !aws configure get region
create_role_url = f'https://console.aws.amazon.com/iam/home?region={region[0]}#/roles$new?step=type'
print(create_role_url)

Then I selected 'SageMaker' and clicked the 'Next: Permissions' button. This showed me I had selected a 'AmazonSageMakerFullAccess' policy, I then clicked the 'Next: Review' button. This shows me a page asking for the Role name, I named it "AmazonSageMaker-ExecutionRole" and then clicked 'Create role'.

The IAM role this method produced looks like this

In [ ]:
account = !aws sts get-caller-identity --query Account --output text
role = f'arn:aws:iam::{account[0]}:role/AmazonSageMaker-ExecutionRole'
print(role)

**Container** 

The naming conventioin for docker images in Amazon's [Elastic Container Registry](https://aws.amazon.com/ecr/) combine your account number and a geographic region with the local container name like this:

In [ ]:
ecr_name = f'{account[0]}.dkr.ecr.{region[0]}.amazonaws.com/{name}:latest'
print(ecr_name)

**SageMaker SDK**

To communicate with SageMaker from Python let's use the official client.

In [ ]:
import boto3
sm = boto3.client('sagemaker')

When we invoke client functions we get a response back from Amazon.

In [ ]:
# create a SageMaker model..
create_model_response = sm.create_model(
    ModelName=name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': ecr_name
    })

# check that worked
print(create_model_response['ModelArn'])

We define the compute resources that are available to an algorithm by 'configuring the endpoint'. For example, we can define which [instance type](https://aws.amazon.com/sagemaker/pricing/instance-types/) we want.

In [ ]:
# instance type (some examples)
cpu_instance = 'ml.m4.xlarge' # $0.311 per hour
gpu_instance = 'ml.p2.xlarge' # $1.361 per hour

# configuration name
config_name = 'byom-config-cpu'

# create configuration
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=config_name,
    ProductionVariants=[{
        'InstanceType': cpu_instance,
        'InitialInstanceCount': 1,
        'ModelName': name,
        'VariantName': 'AllTraffic'}])

print(create_endpoint_config_response['EndpointConfigArn'])

We can now create the endpoint(s)

In [ ]:
# endpoint name
endpoint_name = name+'-endpoint'

# create endpoint
create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=config_name)

print(create_endpoint_response['EndpointArn'])

They take a while to deploy, let's check the status

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

and keep checking until it is done

In [ ]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)

resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

if status != 'InService':
    raise Exception('Endpoint creation did not succeed')

## Test the endpoint


To invoke the endpoint from Python we use the Amazon [SageMaker Runtime](https://docs.aws.amazon.com/sagemaker/latest/dg/API_Operations_Amazon_SageMaker_Runtime.html).

In [ ]:
runtime = boto3.Session().client('runtime.sagemaker')

In [ ]:
with open('../images/dog.jpg', 'rb') as file:
    image = file.read()

In [ ]:
image?

In [ ]:
import matplotlib.image as mpimg

img = mpimg.imread('../images/dog.jpg', 'rb')
print(type(img))

In [ ]:
# import json
# payload = json.dumps({"image": img})

[so thread](https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable)


In [ ]:
import json 
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:


a = np.array([1, 2, 3])

print(json.dumps({'aa': [2, (2, 3, 4), a], 'bb': [2]}, cls=NumpyEncoder))


In [ ]:
# response = runtime.invoke_endpoint(
#     EndpointName=endpoint_name,
#     ContentType='image/jpeg',
#     Body=payload
# )

# result = json.loads(response['Body'].read().decode())

# print(result)

## The End

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
import requests

In [ ]:
r = requests.get('http://127.0.0.1:5000/')
r.text

In [ ]:
r = requests.get('http://127.0.0.1:5000/invocations')
r.text

In [ ]:
r.headers

In [ ]:
import json
url = 'http://127.0.0.1:5000/invocations'
payload = {'image': img.tolist()}
headers = {'content-type': 'application/json'}
 
r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.status_code)
print(r.text)